# Preview Support Tickets

In [ ]:
from snowflake.snowpark.context import get_active_session
import snowflake.snowpark.functions as F
import streamlit as st
session = get_active_session()

In [ ]:
use database GENAI_MODULE3_FINETUNE;
use schema DATA;

In [ ]:
df_support_tickets = session.table('SUPPORT_TICKETS')
df_support_tickets

## Custom email or text reponse generation for support tickets using LLMs

In [ ]:
prompt = """Please write an email or text promoting a new plan that will save customers total costs. 
Also resolve the customer issue based on the ticket category. 
If the contact_preference is text message, write text message response in less than 25 words. 
If the contact_preference is email, write email response in maximum 100 words.
Write only email or text message response based on the contact_preference for every customer.
"""

In [ ]:
ticket_categories = ['Roaming fees', 'Slow data speed', 'Lost phone', 'Add new line', 'Closing account']

## Mistral-7b response

In [ ]:
df_mistral_7b_response_sql = f""" select ticket_id, 
snowflake.cortex.classify_text(request, {ticket_categories}) as ticket_category,
contact_preference, 
trim(snowflake.cortex.complete('mistral-7b',concat('{prompt}', request, ticket_category, contact_preference)),'\n') 
    as mistral_7b_response
from SUPPORT_TICKETS"""

df_mistral_7b_response = session.sql(df_mistral_7b_response_sql)
df_mistral_7b_response

## mistral-large response

In [ ]:
mistral_large_response_sql = f""" select ticket_id, 
snowflake.cortex.classify_text(request, {ticket_categories}) as ticket_category,
contact_preference, 
trim(snowflake.cortex.complete('mistral-large',concat('{prompt}', request, ticket_category, contact_preference)),'\n') 
    as mistral_large_response
from SUPPORT_TICKETS"""

df_mistral_large_response = session.sql(mistral_large_response_sql)
df_mistral_large_response

# Generate dataset to fine-tune mistral-7b

In [ ]:
# Stage 1: Filter by CONTACT_PREFERENCE
df_text = df_mistral_large_response.filter(F.col("CONTACT_PREFERENCE") == 'Text Message')
df_email = df_mistral_large_response.filter(F.col("CONTACT_PREFERENCE") == "Email")

# Stage 2: Apply word count logic
df_text_filtered = df_text.filter(F.regexp_count(F.col("MISTRAL_LARGE_RESPONSE"), r" ") <= 25)
df_email_filtered = df_email.filter(F.regexp_count(F.col("MISTRAL_LARGE_RESPONSE"), r" ") > 30)

# Combine the results
df_filtered = df_text_filtered.union(df_email_filtered)

df_filtered

In [ ]:
df_filtered.write.save_as_table("SUPPORT_TICKET_RESPONSES", mode="overwrite")  # "overwrite" can be changed to "append"

# Optional: Show the saved table to verify
saved_df = session.table("SUPPORT_TICKET_RESPONSES")
saved_df

In [ ]:
df_fine_tune = saved_df.with_column("prompt", 
                                    F.concat(F.lit(prompt), 
                                             F.lit(" "), 
                                             F.lit("Contact preference: "),
                                             F.col("contact_preference"), 
                                             F.lit(" "),
                                             F.col("ticket_category")))\
                        .select("ticket_id",
                                "prompt",
                                "mistral_large_response")

df_fine_tune.write.mode('overwrite').save_as_table('support_tickets_finetune_message_style')

# Fine-tune mistral-7b

## Split data into training and evaluation

In [ ]:
train_df, eval_df = session.table("support_tickets_finetune_message_style").random_split(weights=[0.8, 0.2], seed=42)

train_df.write.mode('overwrite').save_as_table('support_tickets_finetune_message_style_train')
eval_df.write.mode('overwrite').save_as_table('support_tickets_finetune_message_style_eval')

In [ ]:
session.table('support_tickets_finetune_message_style_train').show(1)

In [ ]:
session.table('support_tickets_finetune_message_style_eval').show(1)

## *Fine-tune mistral-7b model using Cortex*

In [ ]:
select snowflake.cortex.finetune('CREATE', 
'SUPPORT_MESSAGES_FINETUNED_MISTRAL_7B', 
'mistral-7b', 
'SELECT prompt, mistral_large_response as completion from support_tickets_finetune_message_style_train',
'SELECT prompt, mistral_large_response as completion from support_tickets_finetune_message_style_eval');

### See status of the fine tuning job

In [ ]:
select snowflake.cortex.finetune('DESCRIBE', 'CortexFineTuningWorkflow_6c475d8a-2b7a-4b96-b51c-78a7a2813e9e');

# Inference using fine-tuned model

In [ ]:
fine_tuned_model_name = 'SUPPORT_MESSAGES_FINETUNED_MISTRAL_7B'

sql = f""" select ticket_id, 
snowflake.cortex.classify_text(request, {ticket_categories}) as ticket_category,
contact_preference, 
trim(snowflake.cortex.complete('{fine_tuned_model_name}',concat('{prompt}', request, ticket_category, contact_preference)),'\n') 
    as fine_tuned_mistral_7b_model_response
from SUPPORT_TICKETS"""

df_fine_tuned_mistral_7b_response = session.sql(sql)
df_fine_tuned_mistral_7b_response

# Auto-generate custom emails and text messages

## Streamlit application to auto-generate custom emails and text messages

In [ ]:
st.subheader("Auto-generate custom emails or text messages")

with st.container():
    with st.expander("Edit prompt and select LLM", expanded=True):
        entered_prompt = st.text_area('Prompt',"""Please write an email or text promoting a new plan that will save customers total costs. If the customer requested to be contacted by text message, write text message response in less than 25 words, otherwise write email response in maximum 100 words.""")
    
        with st.container():
            left_col, mid_col, right_col = st.columns(3)
            with left_col:
                selected_category = st.selectbox('Select category',('Roaming fees', 'Closing account', 'Add new line', 'Slow data speed'))
            with mid_col:
                selected_preference = st.selectbox('Select contact preference', ('Text message', 'Email'))
            with right_col:
                selected_llm = st.selectbox('Select LLM',('llama3-8b', 'mistral-7b', 'mistral-large', 'SUPPORT_MESSAGES_FINETUNED_MISTRAL_7B',))

with st.container():
    _,mid_col,_ = st.columns([.4,.3,.3])
    with mid_col:
        generate_template = st.button('Generate messages ⚡',type="primary")

with st.container():
    if generate_template:
        sql = f"""select s.ticket_id, s.customer_name, concat(IFF(s.contact_preference = 'Email', '📩', '📲'), ' ', s.contact_preference) as contact_preference, snowflake.cortex.complete('{selected_llm}',
        concat('{entered_prompt}','Here is the customer information: Name: ',customer_name,', Contact preference: ', contact_preference))
        as llm_response from support_tickets as s join support_tickets_train as t on s.ticket_id = t.ticket_id limit 10"""

        with st.status("In progress...") as status:
            df_llm_response = session.sql(sql).to_pandas()
            st.subheader("LLM-generated emails and text messages")
            for row in df_llm_response.itertuples():
                status.caption(f"Ticket ID: `{row.TICKET_ID}`")
                status.caption(f"To: {row.CUSTOMER_NAME}")
                status.caption(f"Contact through: {row.CONTACT_PREFERENCE}")
                status.markdown(row.LLM_RESPONSE.replace("--", ""))
                status.divider()
            status.update(label="Done!", state="complete", expanded=True)